In [16]:
import numpy as np
from keras.layers import Input, Dense, Embedding, TimeDistributed, Bidirectional, LSTM, merge, concatenate, Dropout
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras import metrics
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import re
from wikipedia2vec import Wikipedia2Vec
import MeCab
import json

from elmoformanylangs import Embedder

import shinra_util as util

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
# Fix ramdom seed.
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)

In [17]:
wiki_sentence_df = pd.read_csv("../data/wikitext_split_sentence.csv", dtype={'_id': str})
wiki_sentence_df.head()

,_id,title,sentence
0,1300364,イソチオシアネート,イソチオシアネート（Isothiocyanate）とは、-N=C=Sという構造を持つ物質の総...
1,1300364,イソチオシアネート,アブラナ科の植物にしばしば含まれるアリルイソチオシアネートはカラシ油に含まれ、辛味の原因とな...
2,1300364,イソチオシアネート,エドマン分解ではアミノ酸の配列の解析に用いられる。
3,1300364,イソチオシアネート,イソチオシアネートは常に炭素原子を求電子中心とする求電子剤として働く。
4,1300364,イソチオシアネート,フェニチルイソチオシアネートやスルフォラファンなどのイソチオシアネートは発癌や腫瘍化を防ぎ、...


In [18]:
with open("../data/compound_train.json", 'r') as f:
    train_raw = json.load(f)['entry']

ids = [str(entry['WikipediaID']) for entry in train_raw]
np.random.shuffle(ids)

train_production_dict = util.train2dict(train_raw, '製造方法')
train_production_df = wiki_sentence_df.loc[wiki_sentence_df._id.isin(ids)]
train_production_df = util.labeling(train_production_df, train_production_dict)

print("Number of data:", len(train_production_df))
print("True:", len(train_production_df[train_production_df.label == True])
      , "\tFalse:", len(train_production_df[train_production_df.label == False]))
train_production_df.head()

Number of data: 8999
True: 593 	False: 8406


,_id,title,sentence,label
8,2662912,ハロン (化合物),ハロン (halon) は、炭化水素の水素原子（一部または全て）がハロゲン原子で置換されたハ...,False
9,2662912,ハロン (化合物),ハロゲン化炭化水素 (halogenated hydrocarbon) が語源で、アメリカ陸...,False
10,2662912,ハロン (化合物),ハロン類 (halons)、ハロン化合物 (halon compounds) ともいう。,False
11,2662912,ハロン (化合物),ハロンに対し、臭素を含まず、ハロゲンがフッ素と塩素のみの化合物を、フロン（クロロフルオロカー...,False
12,2662912,ハロン (化合物),ただし、フロンが日本特有の語であるのに対し、ハロンは国際的に通用する名である。,False


In [19]:
train_df = train_production_df.loc[train_production_df._id.isin(ids[:500])]
validate_df = train_production_df.loc[train_production_df._id.isin(ids[500:])]

print("Number of train rows:", len(train_df))
print("True:", len(train_df[train_df.label == True])
      , "\tFalse:", len(train_df[train_df.label == False]))

print("Number of validete rows:", len(validate_df))
print("True:", len(validate_df[validate_df.label == True])
      , "\tFalse:", len(validate_df[validate_df.label == False]))

Number of train rows: 7576
True: 510 	False: 7066
Number of validete rows: 1423
True: 83 	False: 1340


In [20]:
wiki2vec = Wikipedia2Vec.load('../../model/ja_wiki2vec_100d.pkl')
m = MeCab.Tagger("-Owakati")

In [21]:
WORD_EMBEDDING_DIM = 100
WORD_LSTM_UNIT = 100
ENTRIES_EMBEDDING_DIM = 100
EMBEDDING_DIM = 100

In [74]:
def wakati(s: str):
    return [w for w in m.parse(s).strip().split()]

def sentence2vec(s: str):
    return [_w2v(w) for w in m.parse(s).strip().split()]

def _w2v(w):
    try:
        return wiki2vec.get_word_vector(w).tolist()
    except KeyError:
        return [0.0] * EMBEDDING_DIM

def entry2vec(title: str):
    try:
        return wiki2vec.get_entity_vector(title).tolist()
    except KeyError:
        return [0.0] * EMBEDDING_DIM

def padding(X: np.array):
    padding_seq = \
    pad_sequences(
        X
        , dtype='float32'
        , padding='post'
        , truncating='pre'
        , maxlen=50
    )
    
    return padding_seq

In [75]:
x_train_words = \
padding(
    train_df.sentence.apply(
        lambda x: sentence2vec(x)
    ).tolist()
)

x_train_entries = np.array(train_df.title.apply(lambda x: entry2vec(x)).tolist())

y_train = train_df.label.values

x_valid_words = \
padding(
    validate_df.sentence.apply(
        lambda x: sentence2vec(x)
    ).tolist()
)
x_valid_entries = np.array(validate_df.title.apply(lambda x: entry2vec(x)).tolist())
y_valid = validate_df.label.values

In [73]:
print(x_train_words.shape)
print(x_train_entries.shape)
print(y_train.shape)

(7576, 50, 100)
(7576, 100)
(7576,)


In [76]:
import keras.backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [77]:
word_embeddings = Input(shape=(None, WORD_EMBEDDING_DIM,), dtype='float32')
entries_embeddings = Input(shape=(ENTRIES_EMBEDDING_DIM,), dtype='float32')

l_drop_word = Dropout(0.5)(word_embeddings)
l_lstm = Bidirectional(LSTM(WORD_LSTM_UNIT))(l_drop_word)
#x = concatenate([l_lstm, entries_embeddings])
x = Dropout(0.5)(l_lstm)
x = Dense(100, activation='relu')(x)
x = Dense(50, activation='relu')(x)
pred = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[word_embeddings, entries_embeddings], outputs=pred)

model.compile(optimizer='adam', loss='binary_crossentropy'
              , metrics=[metrics.binary_accuracy, f1])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, None, 100)         0         
_________________________________________________________________
dropout_16 (Dropout)         (None, None, 100)         0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 200)               160800    
_________________________________________________________________
dropout_17 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense_26 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 51        
Total para

In [112]:
names = [weight.name for layer in model.layers for weight in layer.weights]
weights = model.get_weights()

for name, weight in zip(names, weights):
    print(name, weight.shape)

bidirectional_9/forward_lstm_9/kernel:0 (100, 400)
bidirectional_9/forward_lstm_9/recurrent_kernel:0 (100, 400)
bidirectional_9/forward_lstm_9/bias:0 (400,)
bidirectional_9/backward_lstm_9/kernel:0 (100, 400)
bidirectional_9/backward_lstm_9/recurrent_kernel:0 (100, 400)
bidirectional_9/backward_lstm_9/bias:0 (400,)
dense_25/kernel:0 (200, 100)
dense_25/bias:0 (100,)
dense_26/kernel:0 (100, 50)
dense_26/bias:0 (50,)
dense_27/kernel:0 (50, 1)
dense_27/bias:0 (1,)


In [48]:
from keras.utils import plot_model
import pydot
plot_model(model, show_shapes=True, to_file='model3.png')

In [78]:
model.fit(x=[x_train_words, x_train_entries], y=y_train, epochs=40, batch_size=128)

Epoch 1/40
7576/7576 [==============================] - 14s 2ms/step - loss: 0.2904 - binary_accuracy: 0.9114 - f1: 0.0055
Epoch 2/40
7576/7576 [==============================] - 11s 1ms/step - loss: 0.2141 - binary_accuracy: 0.9327 - f1: 0.0000e+00
Epoch 3/40
7576/7576 [==============================] - 12s 2ms/step - loss: 0.1881 - binary_accuracy: 0.9322 - f1: 0.0000e+00
Epoch 4/40
7576/7576 [==============================] - 10s 1ms/step - loss: 0.1794 - binary_accuracy: 0.9329 - f1: 0.0153
Epoch 5/40
7576/7576 [==============================] - 10s 1ms/step - loss: 0.1754 - binary_accuracy: 0.9331 - f1: 0.0197
Epoch 6/40
7576/7576 [==============================] - 10s 1ms/step - loss: 0.1720 - binary_accuracy: 0.9344 - f1: 0.1407
Epoch 7/40
7576/7576 [==============================] - 10s 1ms/step - loss: 0.1609 - binary_accuracy: 0.9355 - f1: 0.2775
Epoch 8/40
7576/7576 [==============================] - 10s 1ms/step - loss: 0.1625 - binary_accuracy: 0.9362 - f1: 0.2535
Epoch 9/

In [84]:
model.evaluate([x_valid_words, x_valid_entries], y_valid)

1423/1423 [==============================] - 1s 840us/step


[0.10054141646033468, 0.9669711876736505, 0.48377312993132054]

In [80]:
predict = model.predict([x_valid_words, x_valid_entries])

In [81]:
validate_df.reset_index(drop=True, inplace=True)

In [82]:
pred_true = validate_df.loc[np.where(predict >= 0.5)[0]]
pred_false = validate_df.loc[np.where(predict < 0.5)[0]]

TP = pred_true[pred_true.label == True].count()[0]
FP = pred_true[pred_true.label == False].count()[0]
TN = pred_false[pred_false.label == False].count()[0]
FN = pred_false[pred_false.label == True].count()[0]

precision = TP / (TP + FP)
recall = TP / (TP + FN)
F1 = 2 * precision * recall / (precision + recall)

In [83]:
print("TP:", TP)
print("FP:", FP)
print("TN:", TN)
print("FN:", FN)
print("Precision:", precision, "\tRecall:", recall, "\tF1:", F1)

TP: 56
FP: 20
TN: 1320
FN: 27
Precision: 0.7368421052631579 	Recall: 0.6746987951807228 	F1: 0.7044025157232704


In [69]:
pred_true.values

array([['1273610', 'フッ化アンモニウム',
        '製法は氷冷したフッ化水素酸にアンモニアを通じて析出させるか、塩化アンモニウムとフッ化ナトリウムの混合物または硫酸アンモニウムとフッ化カルシウムの混合物を加熱し，昇華させて得る。',
        True],
       ['1273610', 'フッ化アンモニウム',
        '製法は直接アンモニアとフッ化水素の反応生成物を加熱固化するか、同酸、塩基水溶液を混合し蒸発濃縮すると得られる。', True],
       ['1273610', 'フッ化アンモニウム', 'また、前述のように正塩を熱分解しても得られる。', True],
       ['1302712', 'テトラヒドロピラン',
        'アルコールをジヒドロピランと反応させるとテトラヒドロピラニルエーテルが得られ、様々な反応からアルコールを保護する。',
        False],
       ['1302712', 'テトラヒドロピラン',
        'テトラヒドロピランの古典的な有機合成法には、ラネー合金によるジヒドロピランへの水素化がある。', True],
       ['1064260', '有機アルミニウム化合物',
        '実験室において、トリアルキルアルミニウム R3Al はメタセシスやトランスメタル化によって合成される。', True],
       ['1064260', '有機アルミニウム化合物',
        'アルキルリチウムやグリニャール試薬とのメタセシス AlCl 3 + 3 BuLi ⟶ Bu 3 Al + 3 LiCl トランスメタル化 2 Al + 3 Ph 2 Hg ⟶ 2 Ph 3 Al + 3 Hg 工業的には、トリメチル、トリエチルなど単純なトリアルキルアルミニウムは直接法によって調製される。',
        True],
       ['1064260', '有機アルミニウム化合物',
        '工業的には、アルケンを重合させポリオレフィンを作る際の触媒として利用される。', False],
       ['32765', 'コハク酸', '琥珀を破砕し、砂浴で蒸留することで得られた。', True],

In [70]:
pred_false[pred_false.label == True].values

array([['1302688', 'テトラフルオロエチレン',
        'クロロホルムはフッ化水素と反応してクロロジフルオロメタンとなり、クロロジフルオロメタンが熱分解してTFEが生成する。',
        True],
       ['1302688', 'テトラフルオロエチレン',
        'CHCl3 + 2 HF → CHClF2 + 2 HCl 2 CHClF2 → C2F4 + 2 HCl 実験室的には、PTFEを減圧下で熱分解して製造する。',
        True],
       ['1586835', 'ピリドキサールリン酸',
        'ピリドキサールキナーゼ(EC 2.7.1.35) によってピリドキサールから合成され、ATP一分子を要する。', True],
       ['3476579', 'シリルエノールエーテル',
        'シリルエノールエーテルは、求核共役付加反応で作られたエノラートを取り込むことで形成することができる。', True],
       ['3476579', 'シリルエノールエーテル', '適切な基質のブルック転位によって、シリルエノールエーテルを合成できる。',
        True],
       ['1668122', '炭酸ルビジウム',
        '3 RbCl + 4 HNO 3 ⟶ 3 RbNO 3 + NOCl + Cl 2 + 2 H 2 O この硝酸ルビジウムを白金皿中で4倍量のシュウ酸と加熱反応させシュウ酸ルビジウムとする。',
        True],
       ['1668122', '炭酸ルビジウム',
        '2 RbNO 3 + 2 H 2 C 2 O 4 ⟶ Rb 2 C 2 O 4 + 2 NO 2 + 2 CO 2 + 2 H 2 O シュウ酸ルビジウムを空気中で焼き、分解すると炭酸ルビジウムが残る。',
        True],
       ['2133417', 'ヘキサナール', '脂肪酸の酸化により生じる。', True],
       ['1654006', '酸化コバルト(III)',
        '酸化コバルト(III)(Cobalt(III) oxide, Co2O3)